## Simple Appose Test

This notebook shows the basics of using Appose - running Python code in a different environment and getting results back. We'll start super simple!

In [1]:
import appose

### Step 1: Connect to Another Environment

First, we tell Appose which environment we want to use. Think of this as "opening a door" to another Python environment.

Here we set the ```env_path``` to be one of our Pixi environments (```microsam_cellposesam```)

In [2]:
env_path = r"..\..\pixi\microsam_cellposesam\.pixi\envs\default"
env = appose.base(env_path).build()

### Step 2: Send Simple Code

Now we write some Python code as a string. This code will run in the other environment, not here!

**What's happening:** We're creating a message to send to the other environment. The key part is `task.outputs["test"] = 117` - this is how we send information back from the other environment to our current notebook.

- `"test"` is just a name we give to our result (like a label on an envelope)
- `117` is the actual data we want to send back

In [10]:
execution_string = f'''
task.outputs["test"]=117
'''

### Step 3: Run It and Get Results

Here's the magic! We create a `task` and send our code to the other environment and get the results back.  The `task` is available in both environments, so when we add key, value pairs (such as "test", 117) to the task outputs in the externel environment, we can read them back in our current environment. 

In [11]:
with env.python() as python:
    task = python.task(execution_string, queue="main")
    task.wait_for()

    if task.error:
        print(f"⚠️  Task error: {task.error}")

for key, value in task.outputs.items():
    print(f"{key}: {value}")

test: 117


### Something More Useful: Check What's Available

First check if torch is in the environment of this notebook... it shouldn't be...

Then let's ask the other environment: "What versions of important libraries do you have?"

In [12]:
try:
    import torch
except ImportError:
    print("⚠️  torch is not installed in the appose environment.")

⚠️  torch is not installed in the appose environment.


### Create more complex execution string

Here we check the versions and availability of several packages and write the info to the `task.outputs` dictionary. 

In [15]:
execution_string = f'''

import torch
import numpy as np

task.outputs["torch_version"]=torch.__version__
task.outputs["numpy_version"]=np.__version__
task.outputs["cuda_available"]=torch.cuda.is_available()
task.outputs["cudnn_available"]=torch.backends.cudnn.is_available()
task.outputs["cudnn_version"]=torch.backends.cudnn.version()

'''

### Get the Environment Info

We run the above code-string in our second environment, we can read back the `task.outputs` dictionary and find out which versions of torch, numpy, cuda and cudnn we have there. 

Note if the second environment isn't set up or connected right we may get errors. 

In [16]:
with env.python() as python:
    task = python.task(execution_string, queue="main")
    task.wait_for()

    if task.error:
        print(f"⚠️  Task error: {task.error}")

for key, value in task.outputs.items():
    print(f"{key}: {value}")

torch_version: 2.6.0
numpy_version: 1.26.4
cuda_available: True
cudnn_available: True
cudnn_version: 91002
